In [1]:
!pip install gradio
!pip install transformers==4.28
!pip install underthesea
!pip install evaluate
!pip install rouge_score
!pip install sentence_transformers

# Load model

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer, MBartForConditionalGeneration, AutoConfig, TrainingArguments, Trainer
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split as tts
import pandas as pd
import os
import json
from datasets import load_dataset
import torch.nn as nn
from copy import deepcopy
from transformers import GenerationConfig

In [3]:
model_path = "vinai/bartpho-word"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
class SoftPrompt(nn.Module):
    def __init__(self, encoder, plm_embed: nn.Embedding, n_prompt: int = 1000,
                 embed_size:int=1024):
        super().__init__()
        self.plm_embed = plm_embed
        #self.encoder = encoder
        self.n_prompt = n_prompt
        self.list_prompts = [nn.parameter.Parameter(torch.randn(embed_size, dtype=torch.float)) for i in range(n_prompt)]
        #for prompt in self.list_prompts:
        #    nn.init.kaiming_uniform_(prompt)
        self.list_prompts = nn.ParameterList(self.list_prompts)
        self.attent = nn.MultiheadAttention(embed_dim=embed_size, num_heads=32, batch_first=True)

    def inject(self, tokens):
        attention_mask = (tokens != 1).float()
        ori = self.plm_embed(tokens)
        features = ori #self.encoder(tokens, attention_mask=attention_mask)[0]
        list_prompts = torch.cat([i.unsqueeze(0) for i in self.list_prompts]).unsqueeze(0).repeat(tokens.size(0), 1, 1)
        features, _ = self.attent(features, list_prompts, list_prompts)
        return features + ori

    def forward(self, tokens):
        return self.inject(tokens)


In [5]:
model.set_input_embeddings(SoftPrompt(deepcopy(model.get_encoder()), model.get_encoder().embed_tokens).to(device))

In [6]:
!git clone https://huggingface.co/OpenHust/open-bart

Cloning into 'open-bart'...
remote: Enumerating objects: 387, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (381/381), done.
remote: Total 387 (delta 149), reused 0 (delta 0), pack-reused 3
Receiving objects: 100% (387/387), 888.70 KiB | 4.08 MiB/s, done.
Resolving deltas: 100% (149/149), done.
Filtering content: 100% (35/35), 1.58 GiB | 30.68 MiB/s, done.


In [7]:
model.load_state_dict(torch.load("/content/open-bart/pytorch_model.bin"))


<All keys matched successfully>

# Load Dataset

In [8]:
dataset = load_dataset(path="OpenHust/vietnamese-summarization", data_files="bio_medicine.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
small= dataset["train"]

train, test = small.train_test_split(train_size=0.8, seed=0).values()
train, dev = train.train_test_split(test_size=0.125, seed=0).values()

# Demo

In [10]:
def generate(inputs, num_returns=1):
    inputs = tokenizer.encode(inputs, return_tensors="pt", max_length = 1024, padding = True, truncation = True).to(device)
    # outputs = model.generate(inputs, max_length = 1024, num_beams = 10, )
    #outputs = model.generate(inputs, generation_config=genConfig)
    outputs = model.generate(inputs, max_length = 1024, num_beams = 5,
                            num_beam_groups = 5, num_return_sequences = num_returns, no_repeat_ngram_size = 3)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [11]:
doc = test[1]["Document"]
trunc = ""
for i in range(len(doc)//100+1):
    trunc += doc[i*100:(i+1)*100] + "\n"

print(trunc)

Bạn nên triệt sản hoặc thiến chó vì nhiều lý do khác nhau. Một trong số đó là sau khi thực hiện thủ 
thuật này thì chó sẽ ít cắn và ngoan ngoãn hơn do sự thay đổi hoocmon trong cơ thể.  Chó sẽ bớt đi l
ang thang hoặc đánh nhau với những chú chó khác. Sau khi thiến, lượng hoocmon testosterone ở chó đực
 sẽ suy giảm, do đó chúng sẽ bớt hung hăng hơn. Là chủ, bạn cần có trách nhiệm giữ chó ở một khu vực
 giới hạn và an toàn. Điều này không chỉ bảo vệ chú chó mà còn giúp giữ an toàn cho mọi người và thú
 cưng khác.  Không thả rông. Giới hạn không gian sinh hoạt của chó sẽ hạn chế nguy cơ chó gặp và đán
h nhau với những chú chó khác. Giữ chó ở khu vực giới hạn cũng sẽ hạn chế hành vi cắn của chó khi đi
 săn. Nếu biết trước hoặc nghi ngờ chó sẽ cắn, bạn nên tránh những tình huống căng thẳng không cần t
hiết, không đưa chó đến những địa điểm mới hoặc quá náo nhiệt, luôn theo dõi hành vi của chó và lập 
tức đưa chó rời đi nơi khác nếu nó có biểu hiện căng thẳng.  Không cho chó tiếp xúc với quá

In [12]:
generate(doc)

'Triệt sản hoặc thiến chó. Giữ chó ở một khu giới hạn và an toàn. Tránh đưa chó đến nơi đông người. huấn luyện chó.'

In [13]:
test[1]["Summary"]

'Triệt sản hoặc thiến chó. Không thả rông hoặc cho chó đi dạo mà không dùng dây dắt. Tránh những tình huống căng thẳng. Đưa chó đến lớp huấn luyện.'

In [14]:
import gradio as gr

def generate(inputs, num_returns=1):
    inputs = tokenizer.encode(inputs, return_tensors="pt", max_length = 1024, padding = True, truncation = True).to(device)
    # outputs = model.generate(inputs, max_length = 1024, num_beams = 10, )
    #outputs = model.generate(inputs, generation_config=genConfig)
    outputs = model.generate(inputs, max_length = 1024, num_beams = 5,
                            num_beam_groups = 5, num_return_sequences = num_returns, no_repeat_ngram_size = 3)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

demo = gr.Interface(
    fn=generate,
    inputs=gr.Textbox(label="Document", lines=2, placeholder="Document"),
    outputs=gr.Textbox(label="Summary"), examples=test[5:10]["Document"],
)
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fb61d2d55bb84279a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
